<a href="https://colab.research.google.com/github/romankht84/ARC/blob/main/RAG_LangChain_using_LLaMA_2_with_Hugging_Face_on_PWC_Private_Directory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notes for Google Colab: In your notebook, go tountime > Change runtime type > Hardware accelerator > GPU > GPU type > T4. You will need ~8GB of GPU RAM for inference and running on CPU is practically impossible.

In [4]:
# Installing the Libraries and PreRequisites
!pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers

In [5]:
# Initialize Text-generation pipeline with Hugging Face transformers for the pretrained Llama-2-7b-chat-hf model
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [6]:
# Begin initializing Hugging Face items using a Hugging Face access token
hf_auth = '<YOUT-TOKEN-GOES-HERE>'
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)

# Enable evaluation mode to allow model inference
model.eval()

print(f"Model loaded on {device}")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1006: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:479: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0


In [7]:
# Tokenization for converting human-readable text to ML-readable token IDs
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

# A stopping criteria for LLMs to stop generating text
stop_list = ['\nHuman:', '\n```\n']
stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids

# Converting the stop_tokens to longTensor objects
import torch
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids

# Defining custom stopping criteria object
from transformers import StoppingCriteria, StoppingCriteriaList
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False
stopping_criteria = StoppingCriteriaList([StopOnTokens()])

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:640: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [8]:
# Initialize the Hugging Face pipeline
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.01,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [9]:
# Implementing HF Pipeline in LangChain with LLaMA 2
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generate_text)
llm(prompt="What is a data warehouse?")

' A data warehouse is a large, centralized repository of data that is used for reporting and analysis. everybody knows what a data warehouse is, right? Wrong! Many people use the term "data warehouse" without really understanding its purpose or how it differs from other types of databases. In this article, we\'ll explore the definition of a data warehouse, its benefits, and some common use cases to help you better understand what a data warehouse is and why it\'s important.\nA data warehouse is a database designed to store data from multiple sources in a single location. It is used to collect, store, and manage large amounts of data from various sources, such as transactional systems, log files, and external data sources. The main purpose of a data warehouse is to provide a unified view of an organization\'s data, making it easier to analyze and report on.\nBenefits of a Data Warehouse:\n1. Improved data quality: By storing all data in one place, a data warehouse can help ensure data c

## Working on a PDF file. PWC contract file in this case.

In [10]:
# Installing some required plugins and configuring the environment
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [11]:
pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 39.6 MB/s eta 0:00:00


In [12]:
pip install pdf2image

In [13]:
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 52.7 MB/s eta 0:00:00


In [14]:
pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 5.8 MB/s eta 0:00:00


In [16]:
# Ingesting Data using Document Loader

from langchain.document_loaders import PyPDFDirectoryLoader

path = "/content/pwc/"

loader = PyPDFDirectoryLoader(path)
documents = loader.load()
print(len(documents))

1482


In [17]:
# We initialize RecursiveCharacterTextSplitter and call it by passing the documents

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [18]:
# Creating Embeddings and Storing in Vector Store
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# storing embeddings in the vector store
vectorstore = FAISS.from_documents(all_splits, embeddings)

In [19]:
# Initializing ConversationalRetrievalChain

from langchain.chains import ConversationalRetrievalChain

chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

In [20]:
chat_history = []
query = "What is the Shell annual revenue for each year?"
result = chain({"question": query, "chat_history": chat_history})
print(result['answer'])

print('\n\nSources:')
for source in result["source_documents"]:
        print(source.metadata['source'])

 The Shell annual revenue for each year can be found in the table provided in the passage. For example, in 2020, Shell's revenue was $180,543 million.


Sources:
/content/pwc/shell-annual-report-2020.pdf
/content/pwc/shell-annual-report-2022.pdf
/content/pwc/shell-annual-report-2021.pdf
/content/pwc/shell_annual_report_2019.pdf


In [21]:
chat_history = []
query = "How does shell manage macroeconomic risks?"
result = chain({"question": query, "chat_history": chat_history})
print(result['answer'])

print('\n\nSources:')
for source in result["source_documents"]:
        print(source.metadata['source'])

 Shell manages macroeconomic risks through a combination of risk assessment, risk appetite frameworks, and risk response measures. The company monitors external developments, risk indicators, learns from incidents, and assurance findings to identify emerging risks. Additionally, Shell uses a risk appetite framework that considers three distinct factors: strategic risk appetite, operational risk appetite, and conduct risk appetite. These factors aim to capture the range and variety of risks affecting Shell, with specific risk appetite parameters identified and monitored for each one. Furthermore, the company has developed a risk management and internal control system that includes standards and manuals, and the code of conduct, which establish requirements and guidance for managing risks consistently across the group.


Sources:
/content/pwc/shell-annual-report-2020.pdf
/content/pwc/shell-annual-report-2022.pdf
/content/pwc/shell-annual-report-2022.pdf
/content/pwc/shell_annual_report_2

In [22]:
chat_history = []
query = "What has shell audited in 2020 get it only from the Shell 2020 report?"
result = chain({"question": query, "chat_history": chat_history})
print(result['answer'])

print('\n\nSources:')
for source in result["source_documents"]:
        print(source.metadata['source'])

 Based on the information provided in the Shell 2020 Annual Report and Accounts, the following were audited:
* Financial statements
* Internal audit remit and function
* Management judgements
* Risk management and internal control systems
* Activities performed by internal audit
* Evaluation of the quality, efficiency, and effectiveness of the internal audit function
* Performance of the Chief Internal Auditor
Note: This answer is based solely on the information provided in the Shell 2020 Annual Report and Accounts, and does not imply any additional or different audits were conducted by Shell or its external auditors.


Sources:
/content/pwc/shell-annual-report-2021.pdf
/content/pwc/shell-annual-report-2021.pdf
/content/pwc/shell-annual-report-2020.pdf
/content/pwc/shell-annual-report-2021.pdf


In [24]:
chat_history = []
query = "How much shell spends on salary each year?"
result = chain({"question": query, "chat_history": chat_history})
print(result['answer'])

print('\n\nSources:')
for source in result["source_documents"]:
        print(source.metadata['source'])

 Shell spent $12,092 million on employee costs in 2021, including $9,038 million on remuneration, $819 million on social security contributions, $1,696 million on retirement benefits, and $539 million on share-based compensation.


Sources:
/content/pwc/shell-annual-report-2021.pdf
/content/pwc/shell_annual_report_2019.pdf
/content/pwc/shell-annual-report-2020.pdf
/content/pwc/shell-annual-report-2022.pdf


In [25]:
chat_history = []
query = "Who was the Chief Executive Officer of Shell in 2019?"
result = chain({"question": query, "chat_history": chat_history})
print(result['answer'])

 According to the text, the Chief Executive Officer of Shell in 2019 was Ben van Beurden.


In [26]:
chat_history = []
query = "Huibert Vigeveno is working in which company?"
result = chain({"question": query, "chat_history": chat_history})
print(result['answer'])

 Huibert Vigeveno works for Shell.


In [27]:
chat_history = [(query, result["answer"])]
query = "What is his role, age and nationality?"
result = chain({"question": query, "chat_history": chat_history})
print(result['answer'])

# We have used personal pronoun(his), instead of 'Huibert Vigeveno' and the model is still able to understand who are we talking about. Notice that we have used the HISTORY object in this case, whic has the information of the previous query and response and that's why the model is able to understand 'his'.

  Of course! According to the text, Huibert Vigeveno is a member of Shell's Board of Directors and serves as Executive Vice President of Integrated Gas. He is 51 years old and Dutch.


In [34]:
chat_history = []
query = "What is Shell share in Asia and Middle East in 2019?"
result = chain({"question": query, "chat_history": chat_history})
print(result['answer'])

print('\n\nSources:')
for source in result["source_documents"]:
        print(source.metadata['source'], ':::::::', source)

 Based on the information provided in the annual report, Shell's share in Asia and Middle East in 2019 is as follows:
Asia:
* Gross acreage: 21,387 thousand acres
* Net acreage: 14,880 thousand acres
* Shell share of gross acreage: 6,540 thousand acres (or 30%)
* Shell share of net acreage: 6,214 thousand acres (or 43%)
Middle East:
* Gross acreage: 6,289 thousand acres
* Net acreage: 6,082 thousand acres
* Shell share of gross acreage: 2,051 thousand acres (or 32%)
* Shell share of net acreage: 5,823 thousand acres (or 40%)
Therefore, in 2019, Shell had a 30% share in the gross acreage and a 43% share in the net acreage in Asia, and a 32% share in the gross acreage and a 40% share in the net acreage in the Middle East.


Sources:
/content/pwc/shell_annual_report_2019.pdf ::::::: page_content='254\nShell  Annual Report and Accounts 2019ACREAGE AND WELLS\nThe tables below reflect acreage and wells of Shell subsidiaries, joint ventures and associates. The term “gross” refers to the total